In [9]:
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
import matplotlib
from scipy.linalg import block_diag
import cvxpy as cp
import torch.nn as nn
from scipy.linalg import block_diag

In [10]:
def lipSDP(weights,alpha=0,beta=1):
    
    num_layers = len(weights)-1
    dim_in = weights[0].shape[1]
    dim_out = weights[-1].shape[0]
    dim_last_hidden = weights[-1].shape[1]
    hidden_dims = [weights[i].shape[0] for i in range(0,num_layers)]
    dims = [dim_in] + hidden_dims + [dim_out]
    num_neurons = sum(hidden_dims)

    # decision vars
    Lambda = cp.Variable((num_neurons,1),nonneg=True)
    T = cp.diag(Lambda)
    rho = cp.Variable((1,1),nonneg=True)

    A = weights[0]
    C = np.bmat([np.zeros((weights[-1].shape[0],dim_in+num_neurons-dim_last_hidden)),weights[-1]])
    D = np.bmat([np.eye(dim_in),np.zeros((dim_in,num_neurons))])

    for i in range(1,num_layers):
        A = block_diag(A,weights[i])

    A = np.bmat([A,np.zeros((A.shape[0],weights[num_layers].shape[1]))])
    B = np.eye(num_neurons)
    B = np.bmat([np.zeros((num_neurons,weights[0].shape[1])),B])
    A_on_B = np.bmat([[A],[B]])

    cons = [A_on_B.T@cp.bmat([[-2*alpha*beta*T,(alpha+beta)*T],[(alpha+beta)*T,-2*T]])@A_on_B+C.T@C-rho*D.T@D<<0]

    prob = cp.Problem(cp.Minimize(rho), cons)

    prob.solve(solver=cp.MOSEK)
    
    return np.sqrt(rho.value)[0][0]

In [11]:
net = nn.Sequential(
          nn.Linear(2,100),
          nn.ReLU(),
          nn.Linear(100,50),
          nn.ReLU(),
          nn.Linear(50,2))

In [12]:
num_layers = int((len(net)-1)/2)
dim_in = net[0].weight.shape[1]
dim_out = net[-1].weight.shape[0]
hidden_dims = [net[2*i].weight.shape[0] for i in range(0,num_layers)]

# network dimensions
dims = [dim_in] + hidden_dims + [dim_out]

# get weights
weights = np.zeros((num_layers+1,), dtype=np.object)
weights[:] = [net[2*i].weight.detach().numpy().astype(np.float64) for i in range(0,num_layers+1)]

In [13]:
LipConstantDeepSDP = lipSDP(weights)
print(LipConstantDeepSDP)

0.8853361097298602


In [14]:
NaiveUpperBound = 1
for i in range(0,len(weights)):
    NaiveUpperBound = NaiveUpperBound*np.linalg.norm(weights[i],ord=2)
    
print(NaiveUpperBound)

2.4099616695084234


In [15]:
NaiveUpperBound/LipConstantDeepSDP

2.722086722797026